*Disclaimer: This tutorial is in **FRENCH** only.*

# Tutoriel Jules Vernes


<img src="images/jules_verne_1.png" style='float: left; width: 30%; margin:10%; margin-top:0%; margin-bottom:0%'><img src="images/jules_verne_2.png" style='float: left; width: 30%; margin:10%; margin-top:0%; margin-bottom:0%'>


**Pré-requis :**

- Ce notebook doit avoir été généré avec le template NLP du projet Gabarit.


- Télécharger le fichier `texts.csv` se trouvant ici (https://github.com/OSS-Pole-Emploi/AI_frameworks/tree/main/gabarit/template_nlp/nlp_data) et le placer dans le répertoire `{{package_name}}-data` du package que vous avez généré.


- **Lancer ce notebook avec un kernel utilisant l'environnement virtuel de votre projet**. Pour ce faire, il suffit de faire : `python -m ipykernel install --user --name=your_venv_name` (une fois votre environnement virtuel activé). Evidemment, le projet généré doit être installé sur cet environnement virtuel. Il peut être nécessaire de rafraîchir votre navigateur pour voir le kernel apparaître.

---
---
---

## 1. Objectifs




- L'objectif principal de ce tutoriel est de vous introduire au projet Open Source `Gabarit`, développé par l’équipe IA de Pôle Emploi.


- Nous allons traiter un cas d'usage "jouet" : la détection automatique de l'auteur d'un texte du XIXème siècle.


- A la fin de ce tutoriel, nous espérons vous donner tous les outils nécessaires pour démarrer rapidement un nouveau projet d'IA, propre et prêt à être industrialisé.

---
---
---



## 2. Principes


- Ce projet a été developpé dans une logique d'accélération des développements de nos projets IA, et dans l'objectif de fournir une base de code commune facilitant le passage en industrialisation de nos modèles.


- Pour ce faire, nous proposons différents **templates** de projets IA :  
</br>  
</br>  
    - Template NLP : classification de données textuelles  
</br>  
</br>  
    
    - Template NUM : classification et régression sur des données numériques  
</br>  
</br>  
    
    - Template VISION : classification d'images et détection d'objets  
</br>  
</br>  
    
    
- L'idée n'est pas de créer un outil clé en main 'low-code', mais bien de fournir une base de code que chaque Data Scientist pourra adapter à son projet. Il aura ainsi le contrôle de tout ce qu'il s'y passe.


- Les templates sont très similaires entre eux, même s'ils ont évidemment chacun leur spécificité.  


- Les projets générés sont composés d'une partie 'package' et d'une partie 'scripts' qui permettent notamment de lancer les entrainements.  


- Le projet est construit de manière à ce que tous les modèles soient '**agnostic**'. De cette manière, on appelera un modèle toujours de la même façon (e.g. model.predict(...)) peu importe qu'il s'agisse d'un modèle Sklearn ou TensorFlow.


- Des démonstrateurs Streamlit sont intégrés aux projets pour pouvoir rapidement faire des présentations à votre métier !

---
---
---

## 3. Entrainement d'un modèle de détection d'auteurs 'out of the box'

On va partir de textes de différents auteurs du XIXème siècle et fabriquer un modèle permettant **d'identifier automatiquement son auteur**.

---

### 3.1 Imports et fonctions utilitaires

In [ ]:
import os
import pandas as pd
import seaborn as sns
from typing import List
import matplotlib.pyplot as plt
from IPython.display import display

from {{package_name}} import utils
from {{package_name}}.models_training import utils_models

---
### 3.2 Chargement des données

In [ ]:
# Certaines fonctions contenues dans utils permettent d'obtenir les chemins vers les répertoires clés du projet
# Ici, on récupère le chemin vers le dossier {{package_name}}-data
data_path = utils.get_data_path()
# On charge les textes pour regarder leur contenu
df = pd.read_csv(os.path.join(data_path, 'texts.csv'), sep=';')

---
### 3.3 Analyse rapide

In [ ]:
# Affichage du nombre de lignes / colonnes
n_rows = df.shape[0]
n_columns = df.shape[1]
print(f"Nombre de lignes : {n_rows}")
print(f"Nombre de colonnes : {n_columns}")

In [ ]:
# Affichage de 10 lignes aléatoires
df.sample(10).head(10)

In [ ]:
# Valeurs manquantes
for col in df.columns:
    nb_missing = df[col].isna().sum()
    print(f"Valeurs manquantes pour la colonne \033[1m{col}\033[0m : {nb_missing} -> {round(nb_missing / n_rows * 100, 2)} %")

In [ ]:
# Analyse de la cible
ax = sns.countplot(x=df['author'])
plt.xticks(rotation=45)
plt.show(block=False)

---

### 3.4 Découpage du jeu de données en train / test

Nous allons tout d'abord **découper notre dataset en deux parties**, un ensemble d'entrainement et un ensemble de test afin de pouvoir entraîner et tester notre modèle (nous ne fabriquons pas d'ensemble de validation pour simplifier la présentation). 

Pour cela, nous allons utiliser les scripts contenus dans le package. Les scripts sont des aides pour effectuer les opérations courantes de Data Science. Rien ne vous empêche de créer vos propres scripts !  

</br>  
</br>  

Pour couper en deux le dataset, procédez de la manière suivante :


- Lancez un terminal et naviguez dans le répertoire de votre projet  


- Activez votre environnement virtuel  

    (e.g. sur unix : `source venv_{{package_name}}/bin/activate`)


- Naviguez dans le répertoire `{{package_name}}-scripts/utils`

    ```bash
    cd {{package_name}}-scripts/utils
    ```


- Appelez le script de découpe du dataset pour réaliser une séparation "stratified" : 

    ```bash
    python 0_split_train_valid_test.py -f texts.csv --perc_train 0.6 --perc_valid 0.0 --perc_test 0.4
    ```
    
    Note : les scripts vont automatiquement chercher à charger les fichiers de données, modèles, etc. directement depuis les répertoires du projet prévus à cet effet (e.g. `{{package_name}}-data`, `{{package_name}}-models`, ...)

</br>  
</br>  

Vous pouvez alors voir que deux fichiers, `texts_train.csv` et `texts_test.csv` ont été créés dans `{{package_name}}-data`. 

</br>  
</br>  

On peut regarder à quoi ressemble ce nouveau jeu de données :

In [ ]:
# Comme nous allons le voir, certains jeux de données peuvent être accompagnés d'une première ligne de metadata.
# On conseil donc d'utiliser la fonction utils.read_csv qui va simplement vérifier la présence de cette ligne
df_train, metadata_train = utils.read_csv(os.path.join(data_path, 'texts_train.csv'))
df_test, metadata_test = utils.read_csv(os.path.join(data_path, 'texts_test.csv'))

print('---------------------')
print('------- TRAIN -------')
print('---------------------')
print(f"Métadonnées train : {metadata_train}")
print('Echantillon :')
display(df_train.sample(5).head(5))
print('---------------------')
print('------- TEST -------')
print('---------------------')
print(f"Métadonnées test : {metadata_test}")
print('Echantillon :')
display(df_test.sample(5).head(5))

Si par hasard, vous avez fait une fausse manipulation et que vous devez regénérer les fichiers de train / test, vous pouvez tout simplement ajouter l'option `--overwrite` pour écraser les fichiers erronés.

---

### 3.5 Preprocessing sur les textes

Nous allons maintenant faire une étape de préprocessing sur les textes afin de les normaliser et de simplifier l'entrainent de nos modèles. Pour cela, nous allons utiliser le script `1_preprocess_data.py` situé dans `{{package_name}}-scripts`.

</br>  
</br>  


Pour préprocesser nos données textuelles :


- Lancez un terminal et naviguez dans le répertoire de votre projet  


- Activez votre environnement virtuel  

    (e.g. sur unix : `source venv_{{package_name}}/bin/activate`)


- Naviguez dans le répertoire `{{package_name}}-scripts`

    ```bash
    cd {{package_name}}-scripts
    ```


- Appelez le script de préprocessing : 

    ```bash
    python 1_preprocess_data.py -f texts_train.csv --input_col text
    ```
    Notes :
    - `--input_col` permet de préciser sur quelle colonne appliquer le preprocessing.
    - On applique le preprocessing uniquement sur le jeu d'entrainement (et le jeu de validation si on en avait un)

</br>  
</br>  

Un nouveau fichier est donc créé dans le répertoire `{{package_name}}-data` : `texts_train_preprocess_P1.csv`.  
Ce fichier est identique au fichier de base, sauf qu'il possède une colonne supplémentaire ('preprocessed_text') qui contient le texte modifié, et une ligne de metadata qui précise quel preprocessing a été appliqué (e.g. `#preprocess_P1`).

Cette métadonnée sera réutilisée par nos modèles pour savoir quel preprocessing doit être appliquée sur une nouvelle entrée en amont de la prédiction.

</br>  
</br>  

On peut regarder à quoi ressemble ce nouveau jeu de données :

In [ ]:
df_train_preprocessed, metadata_train_preprocessed = utils.read_csv(os.path.join(data_path, 'texts_train_preprocess_P1.csv'))
print('------------------------------------')
print('------- TRAIN - PREPROCESSED -------')
print('------------------------------------')
print(f"Métadonnées train - preprocessed : {metadata_train_preprocessed}")
print('Echantillon :')
display(df_train_preprocessed.head(5))

Le préprocessing appliqué est celui fourni de base par le package mais il est très facile d'ajouter ses propres pipelines de preprocessing.

Pour cela :

- Ouvrez le fichier : `{{package_name}}/preprocessing/preprocess.py`.  


- Dans ce module, vous voyez que l'ensemble des pipelines de preprocessing sont renseignées dans la fonction `get_preprocessors_dict`. C'est ici que vous pouvez rajouter vos pipelines personnalisées.  


- On peut également jeter un coup d'oeil à la fonction `preprocess_sentence_P1` qui indique les différentes étapes de préprocessing (pour plus d'informations, il suffit de lire la documentation du package words_n_fun, une librairie facilitant tout le travail de manipulation de texte).

- Si vous avez l'oeil, vous pouvez remarquer que certains accents n'ont pas été traités correctement par la pipeline `preprocess_sentence_P1`. On va donc corriger cela. Pour ce faire, il suffit de rajouter `'remove_accents'` en première étape dans la pipeline de la fonction `preprocess_sentence_P1`. Puis relancez :

    ```bash
    python 1_preprocess_data.py -f texts_train.csv --input_col text
    ```
    Note : Par défaut, ce script écrase les anciennes versions (comportement susceptible de changer dans le futur).

</br>  
</br>  


Les accents sont maintenant correctement traités comme vous pouvez le vérifiez en utilisant la cellule ci-dessous.  


In [ ]:
df_train_preprocessed, metadata_train_preprocessed = utils.read_csv(os.path.join(data_path, 'texts_train_preprocess_P1.csv'))
print('------------------------------------')
print('------- TRAIN - PREPROCESSED -------')
print('------------------------------------')
print(f"Métadonnées train - preprocessed : {metadata_train_preprocessed}")
print('Echantillon :')
display(df_train_preprocessed.head(5))

Notez que, de manière générale, il est **fortement conseillé de créer une nouvelle pipeline de préprocessing plutôt que d'en modifier une existante**.  
En effet, si un de vos anciens modèles utilisait cette pipeline, ses performances vont changer car il utilisera maintenant la nouvelle (qui a le même nom!).  
Comme nous sommes ici au début d'un projet et que nous n'avons pas encore entraîné de modèles, nous pouvons modifier la pipeline de préprocessing.

N.B. : Ce comportement sera certainement amélioré dans les prochaines mises à jour

---

### 3.6. Entrainement d'un premier modèle

Nous allons maintenant entraîner un premier modèle en utilisant le script `2_training.py`

Pour cela :


- Lancez un terminal et naviguez dans le répertoire de votre projet  


- Activez votre environnement virtuel  

    (e.g. sur unix : `source venv_{{package_name}}/bin/activate`)


- Naviguez dans le répertoire `{{package_name}}-scripts`

    ```bash
    cd {{package_name}}-scripts
    ```


- Appelez le script d'entrainement : 

    ```bash
    python 2_training.py -f texts_train_preprocess_P1.csv --x_col preprocessed_text --y_col author
    ```
    Note :
    - Par défaut, le modèle entrainé sera un simple TF-IDF + SVM sur les données
    - Plusieurs autres modèles sont proposés, il suffit d'aller commenter / décommenter les lignes correspondantes dans le fichier d'entrainement `2_training.py`.

Nous pourrions utiliser ensuite le script `3_predict.py` pour utiliser le modèle pour prédire sur notre ensemble de test mais nous allons procéder autrement pour avoir les résultats directement dans ce notebook. Si vous allez dans `{{package_name}}-models/model_tfidf_svm` vous allez voir un dossier du type `model_tfidf_svm_{YYYY_MM_DD-hh_mm_ss}` qui contient la sauvegarde du modèle que nous venons d'entraîner. 

Vous pouvez par exemple voir le f1_score sur l'ensemble d'entrainement, la matrice de confusion sur l'ensemble d'entraînement etc. Notez que nous aurions pu spécifier un dataset de validation au script `2_training.py` et nous aurions alors aussi accès à des métriques sur ce dataset. 

Ce dossier contient également les fichiers de sauvegarde du modèle que nous allons charger dans la cellule suivante (en remplacant bien sûr `{YYYY_MM_DD-hh_mm_ss}` par les nombres correspondant

In [ ]:
# Chargement du modèle entrainé
model, model_conf = utils_models.load_model('model_tfidf_svm_{YYYY_MM_DD-hh_mm_ss}')

Voilà, le modèle est chargé, prêt à être utilisé. Ce que nous allons directement faire sur l'ensemble de test chargé plus haut :

In [ ]:
df_test_preprocessed['prediction'] = model.predict(df_test_preprocessed['preprocessed_text'])
print('accuracy :', len(df_test_preprocessed[df_test_preprocessed['prediction']==df_test_preprocessed['author']])/len(df_test_preprocessed))

Mais, il est tout nul ce modèle !!!!

Pour remédier à ça, nous pourrions essayer d'autres modèles qui sont déjà packagés dans le projet. 

Mais nous allons utiliser une autre manière de faire qui illustre toute la puissance de ces frameworks. Nous allons créer notre propre modèle qui incorporera tous les à-côtés que nous avons vu (sauvegarde automatique des modèles, chargement des modèles, calcul des métriques...). 

La logique que nous allons appliquer est simple. Au lieu d'appliquer un TF-IDF + SVM à des livres entiers, nous allons découper les livres en 'phrases' et faire apprendre le modèle sur ces 'phrases'. Pour prédire l'auteur d'un livre, nous allons donc découper en 'phrases', prédire pour chaque 'phrase' un auteur et prendre pour prédiction finale, l'auteur qui a le plus de 'phrases' lui correspondant.

On doit donc créer d'abord une fonction qui prend un texte et qui le découpe en phrase. On va également lui indiquer combien de mots par phrase on souhaite. Compléter la fonction suivante :

In [ ]:
def text_to_sentences(text: str, nb_word_sentence: int) -> List[str]:
    '''Transforms a text in sentences.

    Args:
        text (str) : The text to cut in sentences
        nb_word_sentence (int) : The number of words in a sentence
    Returns:
        list : A list of sentences
    
    '''
    

Une fois la fonction ci-dessus complétée, vous pouvez ouvrir le fichier `utils_tutorial_fr.py` situé au même endroit que ce notebook. A l'intérieur, vous trouverez une solution possible pour la fonction ci-dessus (vous pouvez la remplacer par votre version si vous le souhaitez). 

Le reste du fichier contient notre modèle de prédiction d'auteurs: 

La fonction `df_texts_to_df_sentences` permet de formater une liste de textes et d'auteur en un dataframe de 'phrases' (en utilisant la fonction `text_to_sentences`). Il s'agit uniquement d'un fonction de formattage et n'apporte aucune réelle plus-value.

La classe `ModelAuthor` est la classe implémentant notre modèle. Notez qu'elle descend de la classe `ModelTfidfSvm`, ce qui va nous permettre très simplement d'avoir accès à toutes les méthodes voulues. 

Dans l'`__init__`, on rajoute simplement `nb_word_sentence`, l'argument de la fonction ci-dessus.

Dans le `fit`, on ajoute simplement le découpage en 'phrases' avant d'appeler le même modèle TF-IDF+SVM que précedemment.

Dans le `get_nb_sentences_author` on récupère le nombre de 'phrases' attribuées à chaque auteur

Dans le `predict`, suivant que l'on veut les probabilités ou non, soit on donne le pourcentage de 'phrases' associées à chaque auteur ou le nom de l'auteur qui a le plus de 'phrases' associées.

Ainsi en moins d'une centaine de ligne de code, nous avons développé un nouveau modèle qui s'incorpore totalement dans le projet et qui va pouvoir utiliser tous ses outils

Nous allons donc utiliser notre nouveau modèle en faisant les mêmes étapes que précedemment (sauf les étapes de préprocessing du texte qui ne changent pas). 

On va tout d'abord incorporez notre modèle au projet. Pour ce faire, copier le fichier `utils_tutorial_fr.py` à l'endroit où se trouve tous les autres modèles : `{{package_name}}/models_training/`

Il suffit maintenant de modifier le script `2_training.py` pour rajouter notre modèle. En haut du script, rajouter `from {{package_name}}.models_training.utils_tutorial_fr import ModelAuthor` pour importer notre modèle. Et à la ligne 227, mettez en commentaire le modèle `ModelTfidfSvm` et à la place ajoutez le contenu de la cellule suivante:                                              

In [ ]:
model = ModelAuthor(x_col=x_col, 
                    y_col=y_col, 
                    level_save=level_save,
                    multi_label=multi_label,
                    nb_word_sentence=300)

Relancez le script python avec la même commande que précedemment : `python 2_training.py -f texts_train_preprocess_P1.csv --x_col preprocessed_text --y_col author`

Et rechargez le nouveau modèle entrainé avec la cellule suivante (toujours en changeant le nom du modèle chargé):

In [ ]:
model, model_conf = utils_models.load_model('model_author_{YYYY_MM_DD-hh_mm_ss}')

On teste à nouveau notre modèle pour voir sa performance

In [ ]:
df_test_preprocessed['prediction'] = model.predict(df_test_preprocessed['preprocessed_text'])
print('accuracy :', len(df_test_preprocessed[df_test_preprocessed['prediction']==df_test_preprocessed['author']])/len(df_test_preprocessed))

C'est beaucoup mieux!!

Ce petit exemple illustre la manière d'utiliser les frameworks. On part d'un projet contenant tous les outils pour développer rapidement un modèle. On teste quelques modèles inclus pour voir si un correspond à notre cas d'usage. Et, si besoin, on développe son propre modèle en héritant des classes définies dans le projet pour profiter de toutes les fonctions incluses dedans.

Nous allons finalement utiliser un autre outil incorporé dans les frameworks : le démonstrateur. Il est souvent nécessaire de montrer comment fonctionne un modèle à des personnes qui ne l'ont pas développé et le démonstrateur est fait pour cela. Les frameworks utilisent streamlit pour mettre en place ce démonstrateur.

Il suffit de se placer dans le dossier `{{package_name}}/{{package_name}}-scripts/` et de faire `streamlit run 4_demonstrator.py` pour lancer le démonstrateur (il faut peut-être cliquer sur un lien ou le copier-coller dans un navigateur suivant votre configuration).

Dans le menu de gauche, vous pouvez sélectionner le modèle que vous voulez considérer, soit le premier modèle TF-IDF-SVM que nous avons entrainé soit celui utilisant le découpage en phrase. Il suffit ensuite d'entrer le texte que vous voulez tester et appuyer sur predict pour accéder à la prédiction du modèle.